In [50]:
import numpy as np
import networkx as nx
import aocd
from copy import copy
from string import ascii_lowercase as keys
from string import ascii_uppercase as doors
from collections import defaultdict

def create(d) -> nx.Graph:
    if type(d) == str:  # support string and numpy input
        d = np.array(list(map(list,d.split("\n"))))
    g = nx.Graph()
    for spos, src in np.ndenumerate(d):
        for p in [[1,0], [-1,0], [0, 1], [0, -1]]:
            dpos = tuple(np.array(spos)+p)
            if dpos[0] in range(d.shape[0]) and dpos[1] in range(d.shape[1]):
                dest = d[dpos]
                if src != "#" and dest != "#":
                    g.add_edge(spos, dpos, weight=1)
                    if src in keys:    g.nodes[spos]["key"] = src
                    elif src in doors: g.nodes[spos]["door"] = src.lower()
                    g.nodes[spos]["typ"] = src
    
    # trim graph
    for node in g.copy().nodes:
        neigh = list(g.neighbors(node))
        if len(neigh)==2 and g.nodes[node]["typ"] == ".":
            weight = sum([g.edges[(node, n)]["weight"] for n in neigh])
            g.remove_node(node)
            g.add_edge(*neigh, weight=weight)
    
    return g

def bfs(data):
    # init data
    g = create(data)
    startpos = [pos for pos, typ in nx.get_node_attributes(g, "typ").items() if typ=="@"][0]
    jobs = [((0, 0, startpos, frozenset()))]
    pos_door = nx.get_node_attributes(g, "door")
    pos_key =  nx.get_node_attributes(g, "key")
    
    # needed for part 2: just ignore doors that don't have keys in this sub-maze. doesn't affect part 1
    for p, d in list(pos_door.items()):
        if d not in pos_key.values():
            pos_door.pop(p)
    
    # build dict (src) -> (dest, doors_in_way, keys_in_way)
    key_dist = defaultdict(list)
    for p1, k1 in list(pos_key.items())+[[startpos, "@"]]:
        for p2, k2 in pos_key.items():
            if k1 != k2:
                # calculate shortest path between any points and if any any keys/doors are on the way
                sp = nx.shortest_path(g, p1, p2, weight="weight")
                blocked_by = set(sp) & (set(pos_door))
                blocked_by = set([pos_door[bb] for bb in blocked_by])
                
                blocking = set(sp) & set(pos_key) - set([p1,p2])
                blocking = set([pos_key[bb] for bb in blocking])
                
                dist = nx.shortest_path_length(g, p1, p2, weight="weight")
                key_dist[k1].append([k2, dist, blocked_by, blocking])
    
    # assumption: we can't "walk around doors"
    # and the shortest_path(shortest_path(keys)) is shorter 
    jobs = {(0, "@", frozenset())}
    for _ in range(len(pos_key)):  # iterate bfs for number of keys, as we need that many steps
        jobs = list(set(  # unique
            [(steps+dist, target, keys_hold|set([target]))  # add new dist-target-visited tuples
                for steps, start, keys_hold in jobs         # for all current start-tuples
                    for target, dist, blocked_by, blocking in key_dist[start]  # for all target-tuples
                        if target not in keys_hold          # if target is not already reached
                        and not blocked_by - keys_hold      # not blocked by a door without a key
                        and not blocking - keys_hold]))     # and not blocked by a key on the 
    return min([j[0] for j in jobs])

assert bfs("""#########
#b.A.@.a#
#########""")

assert bfs("""########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################""") == 81

# part 1
aocd.submit(bfs(aocd.get_data(day=18)), day=18)

In [51]:
# part 2 assumption: we can act as if all doors that don't have keys in this maze don't exist
# as we can just use another robot and wait in this maze, as we can only move one at a time anyways
data = aocd.get_data(day=18)
d = np.array(list(map(list,data.split("\n"))))  # parse data
center=np.where(d=="@")[0][0]  # square maze -> only the x-coord is enough
d[center-1:center+2, center-1:center+2] = np.array([["@", "#", "@"], ["#", "#", "#"], ["@", "#", "@"]])  # set the center to @#@ ### @#@

# create the 4 sub-mazes
d1 = d[:center+1,:center+1]
d2 = d[center:,:center+1]
d3 = d[:center+1,center:]
d4 = d[center:,center:]

res = sum(map(bfs, [d1,d2,d3,d4]))
aocd.submit(res, day=18)

answer a: 5402
submitting for part b (part a is already completed)
posting 2138 to https://adventofcode.com/2019/day/18/answer (part b) token=...2749


You don't seem to be solving the right level.  Did you already complete it? [Return to Day 18]


<Response [200]>